In [1]:
import tweepy
import numpy as np
import pandas as pd
import psycopg2
import psycopg2.extras
import creds as c
import nltk

In [2]:
# Authorization to consumer key and consumer secret 
auth = tweepy.OAuthHandler(c.consumer_key, c.consumer_secret) 
# Access to user's access key and access secret 
auth.set_access_token(c.access_key, c.access_secret) 
api = tweepy.API(auth) 

tweets = api.user_timeline('tesla', count = 20) 
        
df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns = ['Tweets'])
print(df.head(20))

                                               Tweets
0   RT @elonmusk: Plaid Model S ships next month h...
1   Plaid Model X has the quickest acceleration of...
2   Plaid Model S is the first production car to a...
3   You can play The Witcher 3, Cyberpunk &amp; ma...
4   New Model S &amp; Model X just launched\n\nDet...
5        Also, Model 3 frunks https://t.co/Up848EP3mk
6   Stamping Model Y body parts at our Fremont fac...
7   欢迎! Model Y deliveries in China have officiall...
8   Come work on cell / battery production at Giga...
9   The best crash is no crash — with Autopilot en...
10  Large crumple zones, rigid body structure &amp...
11  Model Y awarded 5-star safety rating in every ...
12  RT @TonyChoMrMagic: I just installed the large...
13  The 7-seater option is now available for order...
14  Model Y Standard Range is now available to ord...
15  In 2020, we produced and delivered half a mill...
16  Order solar panels before the solar tax credit...
17  20k Superchargers and co

In [3]:
df['Tweets']

0     RT @elonmusk: Plaid Model S ships next month h...
1     Plaid Model X has the quickest acceleration of...
2     Plaid Model S is the first production car to a...
3     You can play The Witcher 3, Cyberpunk &amp; ma...
4     New Model S &amp; Model X just launched\n\nDet...
5          Also, Model 3 frunks https://t.co/Up848EP3mk
6     Stamping Model Y body parts at our Fremont fac...
7     欢迎! Model Y deliveries in China have officiall...
8     Come work on cell / battery production at Giga...
9     The best crash is no crash — with Autopilot en...
10    Large crumple zones, rigid body structure &amp...
11    Model Y awarded 5-star safety rating in every ...
12    RT @TonyChoMrMagic: I just installed the large...
13    The 7-seater option is now available for order...
14    Model Y Standard Range is now available to ord...
15    In 2020, we produced and delivered half a mill...
16    Order solar panels before the solar tax credit...
17    20k Superchargers and counting https://t.c

In [4]:
df['cleaned_tweets'] =df['Tweets'].replace("[^a-zA-Z]"," ",regex=True)
df['removing_https'] = df['cleaned_tweets'].str.replace(r'http.*$', '', case=False)
df['lowercase'] = df['removing_https'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['punctuation'] = df['lowercase'].str.replace(r'[^\w\s]', '')


from nltk.corpus import stopwords
stop_words = stopwords.words('english')
df['review_nopunc_nostop'] = df['punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))

from textblob import Word
# Lemmatize final review format
df['lemmatized'] = df['review_nopunc_nostop']\
.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
df['Sentiment'] = df.lemmatized.apply(lambda x: sia.polarity_scores(x)['compound'])
df1 = df[['Tweets','Sentiment']]

In [6]:
param_dic = {
    "host"      : "localhost",
    "database"  : "sentiment_analysis",
    "user"      : "postgres",
    "password"  : "Kumar@711"
}

In [7]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [8]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS tesla_tweets")

#Creating table as per requirement
sql ='''CREATE TABLE tesla_tweets(
   Sno SERIAL NOT NULL,
   Tweets TEXT NOT NULL,
   Sentiment FLOAT NOT NULL
   )'''
cursor.execute(sql)

In [9]:
def fcn(df,table,cur):
    
    
    if len(df) > 0:
        df_columns = list(df)
          # create (col1,col2,...)
        columns = ",".join(df_columns)

          # create VALUES('%s', '%s",...) one '%s' per column
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

           #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)

        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
        conn.commit()
        cur.close()

In [10]:
cur = conn.cursor()
fcn(df1,'tesla_tweets',cur)

In [11]:
cur.execute("select * from tesla_tweets")
output = cur.fetchall()

for i in output:
    print(i[0], end = ". ")
    print("Tweet :", i[1])
    print("Sentiment :", i[2])

1. Tweet : RT @elonmusk: Plaid Model S ships next month https://t.co/HFUPTnQiPB
Sentiment : 0.0
2. Tweet : Plaid Model X has the quickest acceleration of any SUV by far, at 2.5 seconds 0-60
Sentiment : 0.0
3. Tweet : Plaid Model S is the first production car to achieve 0-60 in under 2 seconds
Sentiment : 0.0
4. Tweet : You can play The Witcher 3, Cyberpunk &amp; many more
Sentiment : 0.34
5. Tweet : New Model S &amp; Model X just launched

Details https://t.co/7Ol1BvJoj8
Sentiment : 0.128
6. Tweet : Also, Model 3 frunks https://t.co/Up848EP3mk
Sentiment : 0.0
7. Tweet : Stamping Model Y body parts at our Fremont factory https://t.co/qbLrYowJRC
Sentiment : 0.0
8. Tweet : 欢迎! Model Y deliveries in China have officially begun 🚘🇨🇳 https://t.co/fG5aax1k2b
Sentiment : 0.0
9. Tweet : Come work on cell / battery production at Giga Texas &amp; Giga Berlin! https://t.co/PhsI5bb0uD https://t.co/jEUatMLDmE
Sentiment : 0.0
10. Tweet : The best crash is no crash — with Autopilot enabled, you are 7x 